# Example of fine tunning of an pre-trained AlexNet model on a new dataset for image classification

In [1]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn

cudnn.benchmark = True #-- uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms.
                       #-- If this is set to false, uses some in-built heuristics that might not always be fastest.

cudnn.fastest = True #-- this is like the :fastest() mode for the Convolution modules,
                     #-- simply picks the fastest convolution algorithm, rather than tuning for workspace size

from torch.autograd import Variable
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch.optim as optim

import random, os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from models.alexnet import *
from utils import *

In [2]:
import ConfigParser
Config = ConfigParser.ConfigParser()
Config.read("config.ini")
# path to the dataset
path_to_FOURVIERE_CLEAN2 = Config.get("Corpus", 'path_to_FOURVIERE_CLEAN2')

In [3]:
imSize = 225
batchSize = 32
nb_epoch = 50

In [4]:
# Load train and test data

pathToImg = path_to_FOURVIERE_CLEAN2+"IMAGE_FROM_V/"
# Dictionary with the imgName as key and corresponding class as value for the test and train part
ImgNameToClassTrain = {line.split(' ')[1]:line.split(' ')[0] for line in open(path_to_FOURVIERE_CLEAN2+"/cutTrainTest/u3_train.txt").read().splitlines()}
ImgNameToClassTest = {line.split(' ')[1]:line.split(' ')[0] for line in open(path_to_FOURVIERE_CLEAN2+"/cutTrainTest/u3_test_requestInTrain.txt").read().splitlines()}
# Convert the class to a numeric id
ClassToIdClass = {Artworks:i for i, Artworks in enumerate(sorted(set([line.split(' ')[0] for line in open(path_to_FOURVIERE_CLEAN2+"/cutTrainTest/u3_train.txt").read().splitlines()])))}
# Associate imgName to the corresponding numeric class id
ImgNameToIdClassTrain = {ImgName:ClassToIdClass[ImgNameToClassTrain[ImgName]] for ImgName in  ImgNameToClassTrain.keys()}
ImgNameToIdClassTest = {ImgName:ClassToIdClass[ImgNameToClassTest[ImgName]] for ImgName in  ImgNameToClassTest.keys()}

# open all images
dImgTrain = {imgName:Image.open(pathToImg+imgName) for imgName in ImgNameToClassTrain.keys()}
dImgTest = {imgName:Image.open(pathToImg+imgName).resize((imSize, imSize), Image.BILINEAR) for imgName in ImgNameToClassTest.keys()}

In [5]:
# Compute mean and standard deviation for images normalisation
mean = ComputeMean(dImgTrain.values())
std = ComputeStdDev(dImgTrain.values())

imageTrainTransform = transforms.Compose([
    transforms.Scale(300), 
    transforms.RandomCrop(225), 
    transforms.ToTensor(), 
    transforms.Normalize(mean = mean,
                         std = std),
])

imageTestTransform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(mean = mean,
                         std = std),
])

In [6]:
# define the criterion
criterion = nn.CrossEntropyLoss()

## Fine tune alexNet with pretrained features parameters

In [7]:
# defined a new net with the number of classes corresponding to the dataset
alexTunedClassifier = alexnet(len(ImgNameToIdClassTrain)).train()
# load the pre-trained model and copy only the Features 
copyFeaturesParametersAlexnet(alexTunedClassifier, models.alexnet(pretrained=True))
alexTunedClassifier = alexTunedClassifier.cuda()

copy Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
copy Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
copy Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
copy Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
copy Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [8]:
#define the optimizer to train only the classifier part with lr of 0.01
optimizer=optim.SGD([{'params': alexTunedClassifier.classifier.parameters()},
                     {'params': alexTunedClassifier.features.parameters(), 'lr': 0.0}
                    ], lr=0.01, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(alexTunedClassifier, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(alexTunedClassifier, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)

epoch 0 test : #Correct 162 on 624 (26.0%)
epoch 1 test : #Correct 206 on 624 (33.0%)
epoch 2 test : #Correct 228 on 624 (36.5%)
epoch 3 test : #Correct 230 on 624 (36.9%)
epoch 4 test : #Correct 232 on 624 (37.2%)
epoch 5 test : #Correct 255 on 624 (40.9%)
epoch 6 test : #Correct 233 on 624 (37.3%)
epoch 7 test : #Correct 279 on 624 (44.7%)
epoch 8 test : #Correct 255 on 624 (40.9%)
epoch 9 test : #Correct 244 on 624 (39.1%)
epoch 10 test : #Correct 217 on 624 (34.8%)
epoch 11 test : #Correct 238 on 624 (38.1%)
epoch 12 test : #Correct 231 on 624 (37.0%)
epoch 13 test : #Correct 260 on 624 (41.7%)
epoch 14 test : #Correct 230 on 624 (36.9%)
epoch 15 test : #Correct 253 on 624 (40.5%)
epoch 16 test : #Correct 258 on 624 (41.3%)
epoch 17 test : #Correct 257 on 624 (41.2%)
epoch 18 test : #Correct 269 on 624 (43.1%)
epoch 19 test : #Correct 271 on 624 (43.4%)
epoch 20 test : #Correct 242 on 624 (38.8%)
epoch 21 test : #Correct 239 on 624 (38.3%)
epoch 22 test : #Correct 250 on 624 (40.1%

In [9]:
# save the net
torch.save(alexTunedClassifier, "ckpt/alexFineTuned-50epoch-lr-0.01.ckpt")

In [10]:
# load the model and define the optimizer to train only the classifier part with lr of 0.001
alexTunedClassifier = torch.load("ckpt/alexFineTuned-50epoch-lr-0.01.ckpt")
alexTunedClassifier = alexTunedClassifier.cuda()

optimizer=optim.SGD([{'params': alexTunedClassifier.classifier.parameters()},
                     {'params': alexTunedClassifier.features.parameters(), 'lr': 0.0}
                    ], lr=0.001, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(alexTunedClassifier, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(alexTunedClassifier, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)

epoch 0 test : #Correct 272 on 624 (43.6%)
epoch 1 test : #Correct 282 on 624 (45.2%)
epoch 2 test : #Correct 283 on 624 (45.4%)
epoch 3 test : #Correct 286 on 624 (45.8%)
epoch 4 test : #Correct 284 on 624 (45.5%)
epoch 5 test : #Correct 289 on 624 (46.3%)
epoch 6 test : #Correct 293 on 624 (47.0%)
epoch 7 test : #Correct 294 on 624 (47.1%)
epoch 8 test : #Correct 298 on 624 (47.8%)
epoch 9 test : #Correct 297 on 624 (47.6%)
epoch 10 test : #Correct 293 on 624 (47.0%)
epoch 11 test : #Correct 295 on 624 (47.3%)
epoch 12 test : #Correct 296 on 624 (47.4%)
epoch 13 test : #Correct 300 on 624 (48.1%)
epoch 14 test : #Correct 301 on 624 (48.2%)
epoch 15 test : #Correct 297 on 624 (47.6%)
epoch 16 test : #Correct 298 on 624 (47.8%)
epoch 17 test : #Correct 294 on 624 (47.1%)
epoch 18 test : #Correct 292 on 624 (46.8%)
epoch 19 test : #Correct 290 on 624 (46.5%)
epoch 20 test : #Correct 293 on 624 (47.0%)
epoch 21 test : #Correct 291 on 624 (46.6%)
epoch 22 test : #Correct 297 on 624 (47.6%

In [11]:
# save the net
torch.save(alexTunedClassifier, "ckpt/alexFineTuned-50epoch-lr-0.01-50epoch-lr-0.001.ckpt")

In [13]:
# load the model and define the optimizer to train net with lr of 0.01
alexTunedClassifier = torch.load("ckpt/alexFineTuned-50epoch-lr-0.01-50epoch-lr-0.001.ckpt")
alexTunedClassifier = alexTunedClassifier.cuda()

optimizer=optim.SGD(alexTunedClassifier.parameters(), lr=0.001, momentum=0.9)
for epoch in range(50): # loop over the dataset multiple times
    print "epoch", epoch, 
    trainclassifier(alexTunedClassifier, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(alexTunedClassifier, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)

 epoch 0 test : #Correct 263 on 624 (42.1%)
epoch 1 test : #Correct 229 on 624 (36.7%)
epoch 2 test : #Correct 238 on 624 (38.1%)
epoch 3 test : #Correct 239 on 624 (38.3%)
epoch 4 test : #Correct 240 on 624 (38.5%)
epoch 5 test : #Correct 258 on 624 (41.3%)
epoch 6 test : #Correct 238 on 624 (38.1%)
epoch 7 test : #Correct 260 on 624 (41.7%)
epoch 8 test : #Correct 238 on 624 (38.1%)
epoch 9 test : #Correct 250 on 624 (40.1%)
epoch 10 test : #Correct 268 on 624 (42.9%)
epoch 11 test : #Correct 271 on 624 (43.4%)
epoch 12 test : #Correct 261 on 624 (41.8%)
epoch 13 test : #Correct 254 on 624 (40.7%)
epoch 14 test : #Correct 230 on 624 (36.9%)
epoch 15 test : #Correct 267 on 624 (42.8%)
epoch 16 test : #Correct 250 on 624 (40.1%)
epoch 17 test : #Correct 262 on 624 (42.0%)
epoch 18 test : #Correct 246 on 624 (39.4%)
epoch 19 test : #Correct 243 on 624 (38.9%)
epoch 20 test : #Correct 243 on 624 (38.9%)
epoch 21 test : #Correct 261 on 624 (41.8%)
epoch 22 test : #Correct 244 on 624 (39.1

In [14]:
# save the net
torch.save(alexTunedClassifier, "ckpt/alexFineTuned-50epoch-lr-0.01-50epoch-lr-0.001-50epoch-lr-0.001.ckpt")

In [15]:
# load the model and define the optimizer to train net with lr of 0.001
optimizer=optim.SGD(alexTunedClassifier.parameters(), lr=0.0001, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(alexTunedClassifier, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(alexTunedClassifier, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)

epoch 0 test : #Correct 271 on 624 (43.4%)
epoch 1 test : #Correct 276 on 624 (44.2%)
epoch 2 test : #Correct 276 on 624 (44.2%)
epoch 3 test : #Correct 279 on 624 (44.7%)
epoch 4 test : #Correct 277 on 624 (44.4%)
epoch 5 test : #Correct 282 on 624 (45.2%)
epoch 6 test : #Correct 282 on 624 (45.2%)
epoch 7 test : #Correct 277 on 624 (44.4%)
epoch 8 test : #Correct 283 on 624 (45.4%)
epoch 9 test : #Correct 281 on 624 (45.0%)
epoch 10 test : #Correct 278 on 624 (44.6%)
epoch 11 test : #Correct 279 on 624 (44.7%)
epoch 12 test : #Correct 276 on 624 (44.2%)
epoch 13 test : #Correct 275 on 624 (44.1%)
epoch 14 test : #Correct 272 on 624 (43.6%)
epoch 15 test : #Correct 271 on 624 (43.4%)
epoch 16 test : #Correct 275 on 624 (44.1%)
epoch 17 test : #Correct 272 on 624 (43.6%)
epoch 18 test : #Correct 274 on 624 (43.9%)
epoch 19 test : #Correct 276 on 624 (44.2%)
epoch 20 test : #Correct 273 on 624 (43.8%)
epoch 21 test : #Correct 271 on 624 (43.4%)
epoch 22 test : #Correct 270 on 624 (43.3%

In [16]:
# save the net
torch.save(alexTunedClassifier, "ckpt/alexFineTuned-50epoch-lr-0.01-50epoch-lr-0.001-50epoch-lr-0.001-50epoch-lr-0.0001.ckpt")